# Відповіді на запитання відкритого типу (ODQA або Open-domain question answering)

У цій лабораторній роботі Ви познайомитеся із задачею відповіді на запитання відкритого типу. На відміну від стандартної задачі відповіді на запитання, для якої потребується і запитання і контекст з можливою відповіддю на поставлене запитання, відповідь на запитання відкритого типу потребує лише запитання.
Задача ODQA є досить поширеною, а однією з її варіацій є функція Google Quick (Direct) Answer, що дозволяє отримувати швидкі відповіді на деякі запити прямо на сторінці пошукової видачі ще до переходу на інтернет-ресурс.

<center>
    <figure>
        <img src="https://www.rankranger.com/www-img/web/seo_glossary/google-quick-answer-example.jpg" alt ="Concrete">
        <figcaption>
            <a href="https://www.rankranger.com/www-img/web/seo_glossary/google-quick-answer-example.jpg">What is Google's Direct Answer?</a></figcaption>
    </figure>
</center>

## Архітектура

Існує декілька основних підходів до побудови ODQA-систем, дізнатися більше про які можна за [посиланням](https://lilianweng.github.io/posts/2020-10-29-odqa/).
У рамках цієї роботи Ви побудуєте двоетапне рішення, що складається з пошуковика (**Retriever**) та читача (**Reader**).
- **Retriever** для наданого запиту (запитання) "витягує" зі сховища даних з тисячами чи мільйонами записів релевантні документи, що найбільш ймовірно відповідають на поставлене запитання.
- **Reader** у обраних релевантних документах намагається знайти сегменти (слова, словосполучення або речення), що містять повну відповідь на запитання.

<center>
    <figure>
        <img src="https://lilianweng.github.io/posts/2020-10-29-odqa/QA-retriever-reader.png" alt ="Concrete">
        <figcaption>
            <a href="https://lilianweng.github.io/posts/2020-10-29-odqa/QA-retriever-reader.png">How to Build an Open-Domain Question Answering System? by Lilian Weng</a></figcaption>
    </figure>
</center>

## Підготовка середовища

Процедури тренування моделей потребують значних обчислювальних ресурсів, тому для виконання лабораторної роботи бажано використовувати сервіси з доступом до GPU (Kaggle/Google Colab).

In [ ]:
# Встановлення залежностей
%pip install -U transformers
%pip install -U datasets
%pip install -U evaluate
%pip install -U accelerate
%pip install -U huggingface-hub
%pip install -U sentence-transformers
%pip install -U langchain
%pip install -U langchain-community
%pip install -U langchain-huggingface
%pip install -U faiss-gpu

In [2]:
# виводить інформацію про графічний прискорювач та його поточне навантаження 
!nvidia-smi

Sun Aug 11 21:20:00 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P0             25W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Імпорт бібліотек
import random

import torch
from transformers import pipeline, AutoTokenizer, DefaultDataCollator, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_dataset, Dataset
from evaluate import load
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

In [4]:
# визначення обчислювального пристрою для роботи, cuda використовується для сеансу з обраним графічним прискорювачем
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## 1. Підготовка даних

У лабораторній роботі використовується [частково локалізована версія датасету SQuAD 2.0](https://huggingface.co/datasets/FIdo-AI/ua-squad), що містить тексти, запитання, а також розмічені сегменти у текстах, що є відповідями на запитання. Друга версія датасету відрізняється від першої тим, що деякі запитання можуть не мати відповіді.

In [5]:
# завантаження даних
qa_dataset = load_dataset("FIdo-AI/ua-squad")
qa_dataset, qa_dataset['train']['data'][:3]

Repo card metadata block was not found. Setting CardData to empty.


Generating train split: 0 examples [00:00, ? examples/s]

(DatasetDict({
     train: Dataset({
         features: ['version', 'data'],
         num_rows: 13859
     })
 }),
 [{'Answer': 'виготовлення продуктів та поширення досвіду, які люди хочуть отримати й можуть собі дозволити',
   'Context': '5 січня 2012 року Вест оголосив про створення компанії для творчого контенту DONDA, названої на честь його покійної матері Донди Вест. Під час представлення Вест заявив, що компанія "продовжить там, де зупинився Стів Джобс"; DONDA діятиме як "дизайнерська компанія, яка забезпечить мислителям творчий простір для реалізації своїх мрій та ідей" з "метою виготовлення продуктів та поширення досвіду, які люди хочуть отримати й можуть собі дозволити". Вест, як відомо, мало говорить про діяльність компанії, відсутні як офіційний веб-сайт, так і представлення в соціальних мережах. Креативна філософія DONDA містить необхідність "розміщувати творців у спільному просторі разом із подібними думками", щоб "спростити та естетично вдосконалити все, що ми бачимо, сма

In [6]:
# детермінованість
random.seed(42)

In [7]:
# поділ датасету на тренувальний, валідаційний та тестовий набори
# для запобігання витоку тренувальних даних у валідаційний та тестовий набори
# необхідно ділити саме за контекстами, адже кожний текст може мати декілька запитань
def generate_split(qa_dataset) -> tuple:
    contexts = list(set([x['Context'] for x in qa_dataset['train']["data"]]))
    contexts.sort()
    random.shuffle(contexts)
    train_contexts, val_contexts, test_contexts = set(contexts[:2220]), set(contexts[2220:2420]), set(contexts[2420:])
    return train_contexts, val_contexts, test_contexts

def split_dataset(qa_dataset, train_contexts, val_contexts, test_contexts) -> tuple:
    train_rows, val_rows, test_rows = [], [], []
    for eid, row in enumerate(qa_dataset['train']['data']):
        # пошук індексу початку розміченої відповіді у контексті
        start_idx = row['Context'].find(row["Answer"])
        if start_idx == -1:
            # відповідь не знайдено у тексті (не повинно статися)
            print(row)
            continue
        example = {
            "answers": {"text": [row['Answer']], "answer_start": [start_idx]},
            "context": row['Context'],
            "id": str(eid),
            "question": row['Question']                   
        }
        if not row['Answer']:
            example['answers'] = {"text": [], "answer_start": []}
        if row['Context'] in train_contexts:
            train_rows.append(example)
        elif row['Context'] in val_contexts:
            val_rows.append(example)
        elif row['Context'] in test_contexts:
            test_rows.append(example)
    random.shuffle(train_rows)
    # опціонально, для випадкових прикладів під час виведення даних
    random.shuffle(val_rows)
    random.shuffle(test_rows)
    return train_rows, val_rows, test_rows

train_contexts, val_contexts, test_contexts = generate_split(qa_dataset)
print(len(train_contexts), len(val_contexts), len(test_contexts))

train_rows, val_rows, test_rows = split_dataset(qa_dataset, train_contexts, val_contexts, test_contexts)
len(train_rows), len(val_rows), len(test_rows)

2220 200 200


(11735, 1022, 1102)

In [8]:
train_rows[200:205]

[{'answers': {'text': ['В 1952 році'], 'answer_start': [0]},
  'context': 'В 1952 році Королівською хартією університетський коледж в Хайфілді був модернізований до Саутгемптонського університету. Саутгемптон набув статусу міста, ставши the City of Southampton в 1964 році.',
  'id': '12082',
  'question': 'В якому році Королівська хартія дала назву Саутгемптонському університету?'},
 {'answers': {'text': ['Бен Вестхофф'], 'answer_start': [760]},
  'context': 'Вест є одним з найвідоміших творців ХХІ століття, за свої роботи він отримує схвалення від музичних критиків, шанувальників, колег-музикантів, художників та культурних діячів. Редактор "AllMusic" Джейсон Бірчмайер пише: "В міру того, як його кар\'єра просувалася на початку XXI століття, Вест зруйнував певні стереотипи щодо реперів і став суперзіркою на власних умовах, без пристосування своєї зовнішності, риторики чи музики під будь-які музичні стандарти". Джон Караманік з "New York Times" заявив, що Вест став "громовідводом для су

## 2. Побудова Reader-моделі

Для вирішення задачі відповіді на запитання за контекстом буде використано мережу на базі архітектури кодуючого трансформера, принцип роботи якої полягає у наступному:
- мережа отримує вхідну послідовність, що представляє собою запитання та контекст, розділені спеціальним токеном
- мережа кодує отриману послідовність
- доданий лінійний шар виконує декодування закодованої інформації шляхом оцінки імовірностей бути початком або кінцем сегменту відповіді для кожного токена (слова) з контексту

Фактично, задача визначення сегменту у послідовності дуже схожа на стандартну задачу класифікації, але тут класифікується не уся послідовність, а кожний її елемент.

[Детальніше про кодуючі трансформери](https://jalammar.github.io/illustrated-bert/)

[Детальніше про вирішення задачі класифікації за допомогою кодуючих трансформерів](https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/)

<center>
    <figure>
        <img src="https://mccormickml.com/assets/BERT/SQuAD/input_formatting.png?ref=blog.paperspace.com" alt ="Concrete">
        <figcaption>
            <a href="https://mccormickml.com/assets/BERT/SQuAD/input_formatting.png?ref=blog.paperspace.com">A Comparison of Question Answering Models by Abhijith Neil Abraham</a></figcaption>
    </figure>
</center>

<center>
    <figure>
        <img src="https://blog.paperspace.com/content/images/size/w1000/2020/08/BERT.jpg" alt ="Concrete">
        <figcaption>
            <a href="https://blog.paperspace.com/content/images/size/w1000/2020/08/BERT.jpg">BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding by Jacob Devlin</a></figcaption>
    </figure>
</center>

#### Варіанти для тренування моделі

Варіант = Номер_у_списку % 2 + 1

|**№**| Model                      | Epochs | Learning rate |
|-----|----------------------------|--------|---------------|
|**1**| microsoft/mdeberta-v3-base | 4      | 2e-05         |
|**2**| microsoft/mdeberta-v3-base | 3      | 3e-05         |

In [ ]:
model_name = "microsoft/mdeberta-v3-base"
# завантаження токенізатора
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [10]:
# опис параметрів https://github.com/huggingface/transformers/blob/v4.44.0/examples/pytorch/question-answering/run_qa.py#L105-L206

question_column_name = "question"
context_column_name = "context"
answer_column_name = "answers"
max_seq_length = 384
doc_stride = 128
pad_to_max_length = True
pad_on_right = tokenizer.padding_side == "right"

# підготовка даних для тренування
def prepare_train_features(examples):
    # У деяких питаннях зліва багато пробілів, що може спричинити помилки обрізання послідовностей,
    # тому ми їх видаляємо
    examples[question_column_name] = [q.lstrip() for q in examples[question_column_name]]

    tokenized_examples = tokenizer(
        examples[question_column_name if pad_on_right else context_column_name],
        examples[context_column_name if pad_on_right else question_column_name],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_seq_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length" if pad_to_max_length else False,
    )

    # Оскільки один приклад з довгим контекстом може дати нам кілька ознак, 
    # нам потрібно мати зв'язок між ознакою та відповідним прикладом.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        if tokenizer.cls_token_id in input_ids:
            cls_index = input_ids.index(tokenizer.cls_token_id)
        elif tokenizer.bos_token_id in input_ids:
            cls_index = input_ids.index(tokenizer.bos_token_id)
        else:
            cls_index = 0

        # Отримуємо послідовність, що відповідає цьому прикладу
        sequence_ids = tokenized_examples.sequence_ids(i)

        # Один приклад може мати кілька сегментів
        # отримуємо індекс, що містить цей проміжок тексту.
        sample_index = sample_mapping[i]
        answers = examples[answer_column_name][sample_index]
        # Якщо запитання не має відповіді, то використовуємо спеціальний токен
        # у якості правильної відповіді
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Індекс початку та кінця відповіді в тексті
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Знаходження індексу початку поточного сегменту
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # Знаходження індексу кінця поточного сегменту
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Визначення того, чи входить розмічена відповідь у поточний сегмент
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

# підготовка даних для тестування
def prepare_validation_features(examples):
    examples[question_column_name] = [q.lstrip() for q in examples[question_column_name]]

    tokenized_examples = tokenizer(
        examples[question_column_name if pad_on_right else context_column_name],
        examples[context_column_name if pad_on_right else question_column_name],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_seq_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length" if pad_to_max_length else False,
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [11]:
train_set = Dataset.from_list(train_rows)
val_set = Dataset.from_list(val_rows)
test_set = Dataset.from_list(test_rows)
train_set, val_set, test_set

(Dataset({
     features: ['answers', 'context', 'id', 'question'],
     num_rows: 11735
 }),
 Dataset({
     features: ['answers', 'context', 'id', 'question'],
     num_rows: 1022
 }),
 Dataset({
     features: ['answers', 'context', 'id', 'question'],
     num_rows: 1102
 }))

In [ ]:
# підготовка даних для тренування та тестування
tokenized_train = train_set.map(prepare_train_features, batched=True, remove_columns=train_set.column_names)
tokenized_val = val_set.map(prepare_train_features, batched=True, remove_columns=val_set.column_names)
tokenized_test = test_set.map(prepare_validation_features, batched=True, remove_columns=test_set.column_names)

In [13]:
# завантаження моделі
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
# розміщення моделі на графічному прискорювачі
model.to(device)

pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForQuestionAnswering were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# конфігурація тренера
training_args = TrainingArguments(
    output_dir="deberta_squad_ukr_3epoch_16bs_3e05",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16, # розмір пакету для тренування
    per_device_eval_batch_size=16, # розмір пакету для валідації
    num_train_epochs=3,
    weight_decay=0.01, # L2 регуляризація
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=DefaultDataCollator(),
)

# тренування
# важливо: для запуску тренування у Kaggle необхідно виконати інструкції wandb, що будуть виведені на екран після запуску команди
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.913500,1.070615
2,0.859400,0.966159
3,0.619700,1.044891


TrainOutput(global_step=2346, training_loss=1.0422585528829824, metrics={'train_runtime': 2303.366, 'train_samples_per_second': 16.294, 'train_steps_per_second': 1.019, 'total_flos': 7354983276794880.0, 'train_loss': 1.0422585528829824, 'epoch': 3.0})

In [16]:
# оцінка натренованої моделі
def evaluate_model(eval_set, model, tokenizer, device):
    model.to(device)
    pipe = pipeline("question-answering", model=model, tokenizer=tokenizer, device=device)
    # завантаження метрики, що використовується для оцінки передбачення
    squad_metric = load("squad_v2")

    # exact - точне співпадіння
    # f1 - середня оцінка F1 прогнозованих токенів відповіді у порівнянні із правильною відповіддю
    # детальніше про метрики: https://huggingface.co/spaces/evaluate-metric/squad_v2
    metrics = {"exact": [], "f1": [], "HasAns_f1": [], "HasAns_exact": [], "NoAns_f1": []}

    for idx, row in enumerate(eval_set):
        answer = pipe(question=row['question'], context=row['context'], handle_impossible_answer=True)
        # представлення правильної відповіді
        reference = {"answers": row['answers'], 'id': row['id']}
        # представлення передбаченої відповіді
        prediction = {"id": row['id'], "prediction_text": answer['answer'], "no_answer_probability": 0}
        results = squad_metric.compute(references=[reference], predictions=[prediction])
        # додаємо кожну метрику у список метрик
        for metric_key in metrics.keys():
            if not metric_key in results:
                continue
            metrics[metric_key].append(results[metric_key])
    
        if idx % 100 == 0:
            print(f"Processed {idx} examples")

    # обчислення середнього значення для кожної метрики
    return {key: sum(val) / len(val) if len(val) > 0 else 0 for key, val in metrics.items()}

evaluate_model(test_rows, model, tokenizer, device)

Processed 0 examples


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processed 100 examples
Processed 200 examples
Processed 300 examples
Processed 400 examples
Processed 500 examples
Processed 600 examples
Processed 700 examples
Processed 800 examples
Processed 900 examples
Processed 1000 examples
Processed 1100 examples


{'exact': 61.61524500907441,
 'f1': 74.4583395208912,
 'HasAns_f1': 75.42529825874239,
 'HasAns_exact': 59.50506186726659,
 'NoAns_f1': 70.4225352112676}

## 3. Побудова Retriever-моделі

Для побудови пошукової моделі скористаємося попередньо натренованою моделлю векторних представлень, що для кожної вхідної послідовності формує список чисел. При цьому близькі за сенсом послідовності будуть мати низьке значення для метрик дистанції (або високе для подібності).

[Детальніше про векторні представлення](https://ai.gopubby.com/an-intuitive-101-guide-to-vector-embeddings-ffde295c3558)

[Детальніше про семантичну подібність](https://www.sbert.net/docs/sentence_transformer/usage/semantic_textual_similarity.html)

[Детальніше про семантичний пошук](https://www.sbert.net/examples/applications/semantic-search/README.html)

[Детальніше про семантичний пошук з FAISS](https://huggingface.co/learn/nlp-course/en/chapter5/6)

In [17]:
# побудова сховища даних: збір усіх текстів з датасету
docs = list(set([x['Context'] for x in qa_dataset['train']['data']]))
len(docs), docs[:5]

(2620,
 ['Струнні інструменти класичної епохи - це чотири інструменти, що утворюють струнну частину оркестру: скрипка, віола, віолончель та контрабас. Дерев’яні духові інструменти включали басет-кларнет, басетгорн, кларнет д’амур, класичний кларнет, шалюмо, флейту, гобой і фагот. Клавішні інструменти включали клавікорд та фортепіано. Хоча клавесин все ще використовувався в супроводі basso continuo в 1750-х і 1760-х роках, він вийшов з ужитку в кінці століття. Мідні духові інструменти включали buccin, офіклеїд (замінник серпенту, який був попередником туби) і рожок.',
  'У 1968 році Шварценеггер та його колега-культурист Франко Колумбу почали цегляний бізнес. Бізнес процвітав завдяки маркетинговій кмітливості пари та підвищеному попиті після землетрусу в Сан-Фернандо 1971 року. Шварценеггер і Колумбу використали прибуток від цегляного цеху, щоб розпочати бізнес у сфері пошти, продаючи обладнання для бодібілдингу і фітнесу та навчальні записи.',
  'Музика Шопена була використана в балеті

In [ ]:
# завантаження моделі векторного представлення
encoder = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

# побудова сховища даних: векторизація усіх текстів та організація їх швидкого пошуку
# за допомогою алгоритмів Facebook AI Similarity Search (FAISS)
db = FAISS.from_texts(texts=docs, embedding=encoder)

## 4. ODQA: поєднання Retriever та Reader

In [74]:
def run_odqa(ranker, reader, query, k=10, handle_impossible_answer=True, threshold=0.5, return_all=False):
    # визначення K релевантних текстів для запиту
    docs_and_scores = ranker.similarity_search_with_score(query, k=k)
    query_answer = None
    answer_candidates = []
        
    for doc in docs_and_scores:
        context, retrieval_score = doc
        context = context.page_content
        # визначення сегменту-відповіді у кожному релевантному тексті
        # handle_impossible_answer=True - модель може передбачити відсутність відповіді у тексті
        answer = reader(question=query, context=context, handle_impossible_answer=handle_impossible_answer)
        qa_score = answer['score']
        answer_text = answer['answer']
        answer_candidate = (answer_text, qa_score, context)
        if return_all:
            # зберігаємо усі передбачення, щоб повернути K відповідей
            answer_candidates.append(answer_candidate)
        if qa_score < threshold:
            continue
        if query_answer is not None:
            # визначаємо відповідь з найбільшою ймовірністю
            if query_answer[1] >= qa_score or not answer_text:
                continue
        query_answer = answer_candidate
    return query_answer if not return_all else answer_candidates

# оцінка ефективності усього пайплайну: необхідно не лише отримати релевантний текст, але й знайти у ньому правильну відповідь
def eval_odqa(ranker, reader, test_set, k=10, handle_impossible_answer=True, threshold=0.5):
    results = []
    metrics = {"exact": [], "f1": [], "HasAns_f1": [], "HasAns_exact": [], "NoAns_f1": []}
    squad_metric = load("squad_v2")
    
    for row in test_set:
        query = row['question']
        query_answer = run_odqa(ranker, reader, query, k, handle_impossible_answer, threshold)
            
        reference = {"answers": row['answers'], 'id': row['id']}
        prediction = {"id": row['id'], "prediction_text": "", "no_answer_probability": 0}
        if query_answer is not None:
            prediction["prediction_text"] = query_answer[0]
        metric_scores = squad_metric.compute(references=[reference], predictions=[prediction])
        for metric_key in metrics.keys():
            if not metric_key in metric_scores:
                continue
            metrics[metric_key].append(metric_scores[metric_key])
        
        results.append({"predicted": query_answer, "truth": row})
    return results, {key: sum(val) / len(val) if len(val) > 0 else 0 for key, val in metrics.items()}

In [66]:
reader = pipeline("question-answering", model=model, tokenizer=tokenizer, device=device)

queries = [x['question'] for x in test_rows]
random.shuffle(queries)

# передбачення для 10 випадкових запитів
for query in queries[:10]:
    answer = run_odqa(db, reader, query, k=10, handle_impossible_answer=True, threshold=0.2)
    print(f"question: {query}\nanswer: {answer[0]}\nscore: {answer[1]}\ncontext: {answer[2]}\n\n")

question: Скільки людей за походженням з Пуерто-Рико в 2013 році проживало в Нью-Йорку?
answer:  107 917.
score: 0.5240135788917542
context: У місті, особливо в районі Східного Бостона, є чисельна латиноамериканська громада. Латиноамериканці в Бостоні в основному складаються з пуерториканських (30 506 або 4,9% від загального населення міста), домініканських (25 648 або 4,2% від загального населення міста), сальвадорських (10850 або 1,8% від загального населення міста), колумбійських (6649 або 1,1% від загальної кількості міського населення), мексиканського (5961 або 1,0% від загального населення міста) та гватемальського (4451 або 0,7% від загального населення міста) етнічного походження. Загалом осіб латиноамериканського походження налічується 107 917. У Великому Бостоні ці цифри значно зростають: пуерториканців 175 000+, домініканців 95 000+, сальвадоран 40 000+, гватемальців 31 000+, мексиканців 25 000+ та колумбійців 22 000+.


question: Коли розпочалось відродження класичного вчен

In [69]:
# оцінка точності ODQA пайплайну на тестовому наборі
results, metrics = eval_odqa(db, reader, test_rows, k=5, threshold=0.0)

In [55]:
metrics

{'exact': 34.39201451905626,
 'f1': 40.2248049093357,
 'HasAns_f1': 31.63974691798422,
 'HasAns_exact': 24.409448818897637,
 'NoAns_f1': 76.05633802816901}

In [49]:
results[0:20]

[{'predicted': (' 1903 року,',
   0.7248746156692505,
   "Міст Джорджа Вашингтона - це найбільш завантажений у світі автомобільний міст, який з'єднує Манхеттен з округом Берген, штат Нью-Джерсі. Міст Верразано-Нероу - найдовший підвісний міст в Америці та один із найдовших у світі. Бруклінський міст - це ікона самого міста. Вежі Бруклінського мосту побудовані з вапняку, граніту та цементу Розендейл, а їх архітектурний стиль - неоготичний, з характерними загостреними арками над проходами крізь кам’яні вежі. Цей міст був також найдовшим підвісним мостом у світі з моменту його відкриття до 1903 року, і є першим підвісним мостом із сталевого дроту."),
  'truth': {'answers': {'text': ['1903 року'], 'answer_start': [536]},
   'context': "Міст Джорджа Вашингтона - це найбільш завантажений у світі автомобільний міст, який з'єднує Манхеттен з округом Берген, штат Нью-Джерсі. Міст Верразано-Нероу - найдовший підвісний міст в Америці та один із найдовших у світі. Бруклінський міст - це ікона само

`eval_odqa()` порівнює правильну відповідь лише з найімовірнішою передбаченою, що є справедливим підходом: користувачам важливо отримувати правильний результат на першій позиції. Водночас, важливу роль у оцінюванні пошукових систем можуть відігравати і метрики, що враховують не лише першу позицію, а N передбачень з найбільшими ймовірностями. Наприклад, для двох рішень з приблизно однаковими результатами додаткова оцінка потрапляння правильної відповіді у топ-5 передбачень допоможе визначити рішення з кращою узагальнюючою здібністю.

**Завдання**: модифікуйте функцію `eval_odqa()` так, щоб вона зараховувала правильну відповідь у випадку її знаходження у топ-N (за ймовірністю) передбаченнях. Проведіть повторну оцінку реалізованого рішення та порівняйте результати.

Варіант = Номер_у_списку % 4 + 1

|**Варіант**| N |
|-----------|---|
|**1**      | 3 |
|**2**      | 5 |
|**3**      | 7 |
|**4**      | 10|

In [118]:
def eval_odqa_first_n(ranker, reader, test_set, k=10, handle_impossible_answer=True, threshold=0.5, top_n=5):
    raise NotImplementedError